# データ・サイエンス	データ・サイエンス社会応用論 / ICT社会応用演習Ⅳ
## 14-2. 地震情報を抽出しよう

### 1. 気象庁防災情報XMLの読み込み
#### 1-a) 下準備

In [ ]:
import xml.etree.ElementTree as ET
from dateutil import parser
import requests
import pandas as pd
import time

In [ ]:
EQVOL_LONG_FEED = 'https://www.data.jma.go.jp/developer/xml/feed/eqvol_l.xml'

#### 1-b) フィードのダウンロード

In [ ]:
response = requests.get(EQVOL_LONG_FEED)
response.encoding = 'utf-8'
print(response.text)

#### 1-c) DataFrameの作成

In [ ]:
NS = {'': 'http://www.w3.org/2005/Atom'}

root = ET.fromstring(response.content)

rows = []
for entry in root.findall('entry', NS):
    rows.append({
        'title' : entry.find('title', NS).text,
        'content' : entry.find('content', NS).text,
        'updated' : parser.parse(entry.find('updated', NS).text),
        'link' : entry.find('link', NS).get('href')
    })

df_feed = pd.DataFrame(rows)
df_feed

#### 1-d) 地震情報での絞り込み

In [ ]:
df_eq_feed = df_feed[df_feed['title'] == '震源・震度に関する情報']
df_eq_feed

### 2. 地震情報の取得
#### 2-a) 地震情報のエントリを解析する関数

In [ ]:
def parse_earthquake(eq_feed):
    response = requests.get(eq_feed)
    response.encoding = 'utf-8'
    NS = {
        '': 'http://xml.kishou.go.jp/jmaxml1/body/seismology1/',
        'jmx_eb': 'http://xml.kishou.go.jp/jmaxml1/elementBasis1/'
    }
    
    root = ET.fromstring(response.content)
    earthquake = root.find('Body', NS).find('Earthquake', NS)
    time = parser.parse(earthquake.find('OriginTime', NS).text)
    area = earthquake.find('Hypocenter', NS).find('Area', NS)
    area_name = area.find('Name', NS).text
    area_coord = area.find('jmx_eb:Coordinate', NS).text
    magnitude = earthquake.find('jmx_eb:Magnitude', NS).text

    return {
        'time' : time,
        'area_name': area_name,
        'area_coord': area_coord,
        'magnitude': magnitude
    }

#### 2-b) 地震情報から1件ずつデータを取得

In [ ]:
rows = []
for eq_feed in df_eq_feed['link']:
    rows.append(parse_earthquake(eq_feed))
    time.sleep(0.5)
df_eq = pd.DataFrame(rows)
df_eq